In [1]:
import sys

sys.path.append('../..')

# Step of the underlying

In [2]:
step = 1/360.

# Max maturity of underlying

In [3]:
maturity = 5.

# Data

In [4]:
N = 20000

## Underlyings

In [5]:
from maths.montecarlo.processes.brownianmotions import GeometricBrownianMotion

time_grid = GeometricBrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
drift = 0.02
vol = 0.3

underlyings = [GeometricBrownianMotion(time_grid, x_0, drift, vol) for _ in range(N)]

## Randomization variables

In [6]:
from scipy.stats import expon
 
mean = maturity / 2.
exp_distrib = expon(loc=0, scale=mean)

print "Mean = %s"%mean

Mean = 2.5


In [7]:
from utils import v_time_offseter

In [8]:
zetas = exp_distrib.rvs(size=N)

zetas = v_time_offseter(zetas, time_grid)

print zetas.shape

(20000L,)


## Load intensities of obligors

In [9]:
import dill

raw_intensities = None
with open('../intensities/intensities_no_I5_new.pkl', 'rb') as f:
    raw_intensities = dill.load(f)
    
raw_intensities.keys()

['groups', 'idio']

In [10]:
import numpy as np

raw_idios = np.array(raw_intensities['idio'])
raw_groups = np.array(raw_intensities['groups'])

In [11]:
subsets = []
hazard_rates = []
pillars = []

for d_idio in raw_idios:
    subsets.append(d_idio['id'])
    hazard_rates.append(d_idio['hazard_rates'])
    pillars.append(d_idio['pillars'])
    
for d_groups in raw_groups:    
    subsets.append(d_groups['ids'])
    hazard_rates.append(d_groups['hazard_rates'])
    pillars.append(d_groups['pillars'])

In [12]:
for s, hz, p in zip(subsets, hazard_rates, pillars):
    if len(list(s)) > 1:
        print    
        
    print s, hz, p    

frozenset([0]) [0.0033853684009316522, 0.0048081762873048344] [3, 5]
frozenset([1]) [0.0, 0.0] [3, 5]
frozenset([2]) [0.0, 0.0] [3, 5]
frozenset([3]) [0.0035647268350895425, 0.0049729999603603187] [3, 5]
frozenset([4]) [0.0085808902954963729, -4.3116780252920595e-08] [3, 5]
frozenset([5]) [0.0071488558192109308, 0.0021660883207161351] [3, 5]
frozenset([6]) [0.0, 0.0] [3, 5]
frozenset([7]) [0.0, 0.0] [3, 5]
frozenset([8]) [0.0, 0.0] [3, 5]
frozenset([9]) [0.0053575143772381798, 0.0031357382970320863] [3, 5]
frozenset([10]) [0.0, 0.0] [3, 5]
frozenset([11]) [0.0, 0.0] [3, 5]
frozenset([12]) [0.0, 0.0] [3, 5]
frozenset([13]) [0.00679070303904733, 0.011875018141229196] [3, 5]
frozenset([14]) [0.0, 0.0] [3, 5]
frozenset([15]) [0.0, 0.0] [3, 5]
frozenset([16]) [0.0, 0.0] [3, 5]
frozenset([17]) [0.019527402656596408, 0.02429112885794836] [3, 5]
frozenset([18]) [0.0, 0.0] [3, 5]
frozenset([19]) [0.0, 0.0] [3, 5]
frozenset([20]) [0.0, 0.0] [3, 5]
frozenset([21]) [0.0, 0.0] [3, 5]
frozenset([22]

## Copula

In [13]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, hazard_rates, pillars)

In [14]:
# The last group contains all
# the obligors ids (as a senior CDO tranch)

bc_ids = list(raw_groups[-1]['ids'])
print bc_ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124]


In [15]:
bc_subsets_indexes = []
for id_ in bc_ids:
    bc_subsets_indexes += copula.get_indexes_including(id_)
    
bc_subsets_indexes = list(set(bc_subsets_indexes))
bc_subsets_indexes.sort()

print bc_subsets_indexes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]


## Default times

In [16]:
default_times_mat = copula.generate_default_times(subsets_indexes=bc_subsets_indexes, number=N)

In [17]:
default_times_mat_shifted = v_time_offseter(default_times_mat, time_grid)

In [34]:
index = time_grid.searchsorted(default_times_mat_shifted[:, 0].min())

print time_grid[index-1], time_grid[index-1]+step, time_grid[index]
print default_times_mat[:, 0].min()
print default_times_mat_shifted[:, 0].min()

0.0138888888889 0.0166666666667 0.0166666666667
0.0157264940045
0.0166666666667


# Saving

In [18]:
to_save = {}

to_save["N"] = N
to_save["randomization"] = {
    "distrib": {"loc": 0., "scale": mean},
    "zetas": zetas,
    "maturity": maturity,
}
to_save["credit"] = {
    "copula": copula,
    "bc_subsets_indexes": bc_subsets_indexes,
    "bc_ids": bc_ids,
    "default_times": default_times_mat_shifted,
}
to_save["underlyings"] = underlyings

In [19]:
import dill

with open('data.pkl', 'wb') as f:
    dill.dump(to_save, f)